## Analyzing dendritic data with CaImAn
This notebook shows an example on how to analyze two-photon dendritic data with CaImAn. It closely follows the other notebooks, so if there are bits left unexplained, please see the main demos. 

In [ ]:
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil
from pathlib import Path

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        ipython().magic('load_ext autoreload')
        ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import nb_view_patches
from skimage.util import montage
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()
hv.notebook_extension('bokeh')

## Setting up a logger (optional)
If you want information (printed to file or to console) you can set this up.

In [ ]:
use_logfile = False  # set to true to log to file
if use_logfile:
    log_file = Path(cm.paths.get_tempdir()) / 'cnmf_demo.log' # 
    print(f"Will save logging data to {log_file}")
else:
    log_file = None
log_format = "{asctime} - {levelname} - [{filename} {funcName}() {lineno}] - pid {process} - {message}"
logging.basicConfig(format=log_format,
                    filename=log_file, 
                    level=logging.INFO, style="{") #DEBUG, INFO, WARNING, ERROR, CRITICAL

## Selecting the data

We provide an example file from mouse barrel cortex, courtesy of Clay Lacefield and Randy Bruno, Columbia University. The `download_demo` command will automatically download the file and store it in your caiman_data folder the first time you run it. To use the demo in your own dataset you can set:

```fnames = [/path/to/file(s)]```.

In [ ]:
fnames = [download_demo('data_dendritic.tif')]

## Viewing the data
Prior to analysis you can view the data by setting `display_movie = False`. The example file provided here is small, with FOV size 128 x 128. To view it larger we set `magnification = 4` in the play command. This will result to a movie of resolution 512 x 512. For larger files, you will want to reduce this magnification factor (for memory and display purposes). The movie displays using OpenCV, and will open a separate window: to close it, press `q` on your keyboard when it is the active window.

In [ ]:
display_movie = False
if display_movie:
    m_orig = cm.load_movie_chain(fnames)
    ds_ratio = 0.25
    m_orig.resize(1, 1, ds_ratio).play(
        q_max=99.5, fr=30, magnification=4)

## Parameter setting

First we'll perform motion correction followed by CNMF.

One of the main differences when analyzing dendritic data, is that dendritic segments are not localized and can traverse significant parts of the FOV. They can also be spatially non-contiguous. To capture this property, CaImAn has two methods for initializing CNMF: `sparse_nmf` which uses scikit-learn's built-in non-negative factorization ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)), and `graph-nmf`, where the Laplacian of the pixel affinity matrix is used as a regularizer to promote spatial components that capture pixels that have similar timecourses. These can be selected with the parameter `method_init`.

In this demo we use (and recommend using) the `graph_nmf` initialization method although `sparse_nmf` can also be used, as indicated below.

Since the dataset is small we can run CNMF without splitting the FOV in patches. This can be helpful because of the non-localized nature of the dendritic segments. To do that we set `rf = None`. Using patches is also supported as demonstrated below. Also note, that spatial and temporal downsampling can also be used to reduce the data dimensionality by appropriately modifying the parameters `ssub` and `tsub`. Here they are set to 1 since it is not necessary.

The graph NMF method was originally presented in the paper:

Cai, D., He, X., Han, J., & Huang, T. S. (2010). Graph regularized nonnegative matrix factorization for data representation. IEEE transactions on pattern analysis and machine intelligence, 33(8), 1548-1560.

In [ ]:
# dataset dependent parameters
fr = 30                             # imaging rate in frames per second
decay_time = 0.5                    # Length of a typical transient in seconds (see source/Getting_Started.rst)

# motion correction parameters
strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (6, 6)         # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# parameters for source extraction and deconvolution
p = 0                       # order of the autoregressive system
gnb = 2                     # number of global background components
merge_thr = 0.75            # merging threshold, max correlation allowed
rf =  None                  # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = None          # amount of overlap between the patches in pixels
K = 60                      # number of components per patch
method_init = 'graph_nmf'   # initialization method (you could also use 'sparse_nmf' for dendritic data)
alpha = 0.5                 # sparsity regularizer term (default is 0.5): used only for sparse_nmf
ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

opts_dict = {'fnames': fnames,
            'fr': fr,
            'decay_time': decay_time,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'alpha_snmf': alpha,  # used only for sparse_nmf
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'merge_thr': merge_thr}

opts = params.CNMFParams(params_dict=opts_dict)

In [ ]:
opts.init

## Start cluster

In [ ]:
print(f"You have {psutil.cpu_count()} CPUs available in your current environment")
processors_to_use = None    # Set to number less than cpu_count - 1 if you have memory problems

In [ ]:
# start a cluster for parallel processing 
# note if a cluster already exists it will be closed so a new session will be opened
if 'dview' in locals():  # locals contains list of current local variables
    print('Closing previous cluster')
    cm.stop_server(dview=dview)
print("Setting up new cluster")
c, dview, n_processes = cm.cluster.setup_cluster(backend='multiprocessing', 
                                                 n_processes=processors_to_use, 
                                                 single_thread=False,
                                                 ignore_preexisting=False)
print(f"Successfully set up cluster with {n_processes} processes active")

## Motion correction
First we create a motion correction object and then perform the registration. The provided dataset has already been registered with a different method so we do not expect to see a lot of motion.

In [ ]:
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

In [ ]:
%%time
#%% Run piecewise-rigid motion correction using NoRMCorre
mc.motion_correct(save_movie=True)
m_els = cm.load(mc.fname_tot_els)
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0 
    # maximum shift to be used for trimming against NaNs

## Display registered movie next to the original (optional)

In [ ]:
#%% compare with original movie
display_movie = False
if display_movie:
    m_orig = cm.load_movie_chain(fnames)
    ds_ratio = 0.2
    cm.concatenate([m_orig.resize(1, 1, ds_ratio) - mc.min_mov*mc.nonneg_movie,
                    m_els.resize(1, 1, ds_ratio)], 
                   axis=2).play(fr=60, q_max=99.5, magnification=4, offset=0)  # press q to exit

## Memory mapping

In [ ]:
#%% MEMORY MAPPING
# memory map the file in order 'C'
fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=border_to_0) # exclude borders

# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)

In [ ]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=processors_to_use, single_thread=False)

## Run CNMF with the chosen initialization method
Note that in the parameter setting we have set `only_init = True` so only the initialization will be run.

In [ ]:
%%time
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(images)

## Display components
And compute the correlation image as a reference image

In [ ]:
CI = cm.local_correlations(images[::1].transpose(1,2,0))
CI[np.isnan(CI)] = 0

In [ ]:
cnm.estimates.hv_view_components(img=CI)

## Plot a montage array of all the components

We can also plot a montage of all the identified spatial footprints.

In [ ]:
A = cnm.estimates.A.toarray().reshape(opts.data['dims'] + (-1,), order='F').transpose([2, 0, 1])
Nc = A.shape[0]
grid_shape = (np.ceil(np.sqrt(Nc/2)).astype(int), np.ceil(np.sqrt(Nc*2)).astype(int))
plt.figure(figsize=np.array(grid_shape[::-1])*1.5)
plt.imshow(montage(A, rescale_intensity=True, grid_shape=grid_shape))
plt.title(f'Montage of initial components {alpha=}');
plt.axis('off');

## Run CNMF again to refine the components
The components found by the initialization have captured some structure, but have likely not converged on a solution yet, and are not particularly sparse (especially if you used `graph_nmf`). Using the `refit` command will start with the initial solutions as seeds, and refine them, typically ending up with more sparse solutions.

In [ ]:
cnm2 = cnm.refit(images, dview=dview)

## Now plot the components again

In [ ]:
cnm2.estimates.hv_view_components(img=CI)

In [ ]:
A2 = cnm2.estimates.A.toarray().reshape(opts.data['dims'] + (-1,), order='F').transpose([2, 0, 1])
Nc = A2.shape[0]
grid_shape = (np.ceil(np.sqrt(Nc/2)).astype(int), np.ceil(np.sqrt(Nc*2)).astype(int))
plt.figure(figsize=np.array(grid_shape[::-1])*1.5)
plt.imshow(montage(A2, rescale_intensity=True, grid_shape=grid_shape))
plt.title(f'Montage of refit components {alpha=}');
plt.axis('off');

## Component evaluation
the components are evaluated in three ways:
   a) the shape of each component must be correlated with the data
   b) a minimum peak SNR is required over the length of a transient
   c) each shape passes a CNN based classifier

In [ ]:
min_SNR = 4  # signal to noise ratio for accepting a component
rval_thr = 0.85  # space correlation threshold for accepting a component
cnn_thr = 0.99  # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

cnm2.params.set('quality', {'decay_time': decay_time,
                           'min_SNR': min_SNR,
                           'rval_thr': rval_thr,
                           'use_cnn': False, # do not use cnn for dentritic data
                           'min_cnn_thr': cnn_thr,
                           'cnn_lowest': cnn_lowest})

cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview);

Plot contours of selected and rejected components

In [ ]:
# accepted components
cnm2.estimates.hv_view_components(img=CI, idx=cnm2.estimates.idx_components)

In [ ]:
# rejected components
cnm2.estimates.hv_view_components(img=CI, idx=cnm2.estimates.idx_components_bad)

## Extract DF/F values

In [ ]:
cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)

The resulting components are significantly more sparse and capture a lot of the data structure.

## Play a movie with the results

The movie will show the original, denoised and residual movies in the left, middle, and right panels respectively. 
The flag `include_bck = False` will remove the estimated background from the original and denoised panels. To include it set it to `True`.

In [ ]:
cnm2.estimates.play_movie(images, q_max=99.9,
                          magnification=4,
                          include_bck=False);
# press q to exit

## Save the object
You can save the results of the analysis for future use and/or to load them on the GUI.

In [ ]:
cnm2.estimates.Cn = CI # save the correlation image for displaying in the background
cnm2.save('dendritic_analysis.hdf5')

## Now try CNMF with patches
As mentioned above, the file shown here is small enough to be processed at once. For larger files patches and/or downsampling can be used. We show how to use patches below. Note that unlike in somatic imaging, for dendritic imaging we want the patches to be large and have significant overlap so enough structure of the dendritic segments is captured in the overlapping region. This will help with merging the components. However, a too large patch size can result to memory issues due to large amount of data loaded concurrently onto memory.

In [ ]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(backend='multiprocessing', 
                                                 n_processes=processors_to_use, 
                                                 single_thread=False)

## Change some parameters to enable patches
To enable patches the `rf` parameter should be changed from `None` to some integer value. Moreover the amount of overlap controlled by `stride_cnmf` should also change. Finally, since the parameter `K` specifies components per patch it should be reduced compared to its value when operating on the whole FOV at once. It is important to experiment with different values for these parameters since results can sensitive, due to the non stereotypical shapes and activity patterns of dendrites.

In [ ]:
rf = 48   # size of each patch is (2*rf, 2*rf) 
stride_cnmf = 16
Kp = 25   # reduce the number of components as it is now per patch
opts.change_params({'rf': rf,
                    'stride': stride_cnmf,
                    'K': Kp});

In [ ]:
cnm_patches = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm_patches = cnm_patches.fit(images)

In [ ]:
cnm_patches.estimates.hv_view_components(img=CI)

In [ ]:
Ap = cnm_patches.estimates.A.toarray().reshape(opts.data['dims'] + (-1,), order='F').transpose([2, 0, 1])
Nc = Ap.shape[0]
grid_shape = (np.ceil(np.sqrt(Nc/2)).astype(int), np.ceil(np.sqrt(Nc*2)).astype(int))
plt.figure(figsize=np.array(grid_shape[::-1])*1.5)
plt.imshow(montage(Ap, rescale_intensity=True, grid_shape=grid_shape))
plt.title('Montage of found spatial components (patch run)');
plt.axis('off');

## Refit to sparsify
As before we can pass the results through the CNMF `refit` function that can yield more sparse components.

In [ ]:
cnm_patches2 = cnm_patches.refit(images, dview=dview)

In [ ]:
Ap2 = cnm_patches2.estimates.A.toarray().reshape(opts.data['dims'] + (-1,), order='F').transpose([2, 0, 1])
Nc = Ap2.shape[0]
grid_shape = (np.ceil(np.sqrt(Nc/2)).astype(int), np.ceil(np.sqrt(Nc*2)).astype(int))
plt.figure(figsize=np.array(grid_shape[::-1])*1.5)
plt.imshow(montage(Ap2, rescale_intensity=True, grid_shape=grid_shape))
plt.title('Montage of refit components with patches');
plt.axis('off');

In [ ]:
cnm_patches2.estimates.hv_view_components(img=CI)

In [ ]:
cnm_patches2.estimates.play_movie(images, q_max=99.9,
                                  magnification=4,
                                  include_bck=False);
# press q to exit

## Compare the two approaches
To identify how similar components and the two approaches yield, we can compute the cross-correlation matrix between the spatial footprints derived from the two approaches.

In [ ]:
AA = np.corrcoef(cnm2.estimates.A.toarray(), cnm_patches2.estimates.A.toarray(), rowvar=False)

In [ ]:
plt.imshow(AA[:cnm2.estimates.A.shape[1], cnm2.estimates.A.shape[1]:])
plt.colorbar();
plt.xlabel('with patches')
plt.ylabel('without patches')
plt.title('Correlation coefficients');

## Stop the cluster when you're done

In [ ]:
#%% STOP CLUSTER and clean up log files
cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)